## Com funcionen els models  BERT, RoBERTA, o la seva adaptació al català BERTA?

A continuació es mostra un exemple del funcionament del model BERTA (l'adaptació al català del model RoBERTA), i la tasca *fill-mask*.




---

Frase d'exemple: "Ella és \<mask\>."  
Buscarem les probablitats que retorna el model per les següents paraules:
* actriu
* advocada
* infermera
* doctora
* cambrera
* taxista



---


https://huggingface.co/projecte-aina/roberta-base-ca-v2

### Importar llibreries necessàries

In [1]:
import transformers
import torch
from torch.nn.functional import softmax

# Mostrem les versions utilitzades
print("torch        ", torch.__version__)
print("transformers ", transformers.__version__)

torch         1.12.1
transformers  4.20.1


### Carreguem els models

In [2]:
# Carreguem el tokenizer i el model.
# El primer cop que s'utilitzi el model, es descarregarà de la pàgina oficial i es 
# guardarà a la carpeta local "~/.cache/huggingface/hub"

# https://huggingface.co/projecte-aina/roberta-base-ca-v2

tokenizer = transformers.AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
model = transformers.AutoModelForMaskedLM.from_pretrained("projecte-aina/roberta-base-ca-v2")

### Exemple

In [3]:
text = "Ella és <mask>."

# Tokenizació
input_ids = tokenizer(text, return_tensors="pt").input_ids

# Posició de la màscara
mask_index = input_ids.tolist()[0].index(tokenizer.mask_token_id)

# Obtenim els valors del model amb el text tokenitzat
model_logits = model(input_ids).logits
model_probs = softmax(model_logits, dim=2)


# Mostra el text tokenitzat i la mida
print("Text original:                   ", text)
print("Text tokenitzat:                 ", input_ids.tolist())
print("Posició de la màcara:            ", mask_index)
print("Text tokenitzat (dimensions):    ", list(input_ids.size()))


# Mostrem els valors donats per el model
print("Model logits (dimensions):       ", list(model_probs.shape))
print("Model probabilitats (dimensions):", list(model_probs.shape))


Text original:                    Ella és <mask>.
Text tokenitzat:                  [[0, 11947, 423, 4, 2462, 2]]
Posició de la màcara:             3
Text tokenitzat (dimensions):     [1, 6]
Model logits (dimensions):        [1, 6, 50262]
Model probabilitats (dimensions): [1, 6, 50262]


In [4]:
# Obtenim els 20 valors més probables i els mostrem per pantalla
values, indices = model_probs[0,mask_index].topk(k=20)
for v,i in zip(values.tolist(), indices.tolist()):
    print(f'{i:>6}  {tokenizer.decode(i):<10} {v:.2%}')

  1008   així      4.94%
   331   la        2.63%
  9238   feliç     2.27%
   411   una       1.96%
   603  ...        1.70%
  1811   aquí      1.65%
  1486   catalana  1.31%
 13202   russa     1.23%
   315   de        1.16%
  6230   francesa  0.92%
 13627   perfecta  0.92%
  3074   diferent  0.90%
  3537   allà      0.87%
 14812   morta     0.85%
  2984   ella      0.82%
   742   gran      0.81%
    68  .          0.72%
  9203   alemanya  0.72%
  1629   dona      0.72%
 19461   bella     0.72%


In [5]:
# Busquem les probabilitats per les paraules buscades
print(f'{"paraula":<10}{"token_id":>10}{"prob":>12}')
print("================================")

for paraula in ["actriu", "advocada", "infermera", "doctora", "cambrera", "taxista"]:
    
    # Obtenim el token ID de la paraula (agafem el 2 element).
    
    # S'ha de vigilar amb aquest mètode ja que algunes paraules es codifiquen amb 
    # més d'un valor, però amb les paraules utilitzades amb l'exemple és correcte.

    # Exemple de tokenització amb paraules d'un sol token ("infermera") i paraula multi-token ("infermer")
    # tokenizer.encode("infermera") => [0, 13751, 2]
    # tokenizer.encode("infermer") => [0, 4139, 4653, 2]
    token_id = tokenizer.encode(paraula)[1]
    
    # Obtenim la probabilitat 
    prob = model_probs[0, mask_index, token_id]
    
    
    # Mostrem els resultats
    print(f"{paraula:<10}{token_id:>10}{prob:>12.4%}")


paraula     token_id        prob
actriu         13751     0.2407%
advocada       32308     0.1001%
infermera      26849     0.4912%
doctora        18530     0.1264%
cambrera       41941     0.1268%
taxista        38757     0.0100%
